In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from brisc.manuscript_analysis.flatmap_projection import compute_flatmap_coors

from pathlib import Path
import pandas as pd
import seaborn as sns
import numpy as np
from brisc.manuscript_analysis import distance_between_cells as dist_cells
from brisc.manuscript_analysis import spatial_plots_rabies as spatial
from brisc.manuscript_analysis import barcodes_in_cells as bc_cells
from brisc.manuscript_analysis.load import load_cell_barcode_data

from iss_preprocess.io import get_processed_path

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm

arial_font_path = "/nemo/lab/znamenskiyp/home/shared/resources/fonts/arial.ttf"
arial_prop = fm.FontProperties(fname=arial_font_path)
plt.rcParams["font.family"] = arial_prop.get_name()
plt.rcParams.update({"mathtext.default": "regular"})  # make math mode also Arial
fm.fontManager.addfont(arial_font_path)
matplotlib.rcParams["pdf.fonttype"] = 42  # for pdfs

In [ ]:
barseq_path = get_processed_path("becalia_rabies_barseq").parent.parent
main_path = Path("/nemo/lab/znamenskiyp/")

error_correction_ds_name = "BRAC8498.3e_error_corrected_barcodes_26"
cells_df = load_cell_barcode_data(
    barseq_path / f"processed/becalia_rabies_barseq/BRAC8498.3e/analysis",
    error_correction_ds_name=error_correction_ds_name,
)

relative_presyn_coords, distances, starter_df = (
    dist_cells.determine_presynaptic_distances(cells_df)
)

In [ ]:
flat_coors = compute_flatmap_coors(cells_df, distance_cutoff=150)
cells_df["flatmap_x"] = flat_coors[:, 0]
cells_df["flatmap_y"] = flat_coors[:, 1]
cells_df["flatmap_z"] = flat_coors[:, 2]

In [ ]:
normalised_coors = compute_flatmap_coors(
    cells_df, distance_cutoff=150, thickness_type="normalized_layers"
)
cells_df["normalised_layers"] = np.array(normalised_coors[:, 2])

In [ ]:
for col in ["gene_total_counts", "main_barcode"]:
    if col in cells_df.columns:
        cells_df.drop(col, axis=1, inplace=True)


def match_barcodes(series, barcodes):
    return series.apply(lambda bcs: len(bcs.intersection(barcodes)) > 0)


starters = cells_df[cells_df["is_starter"] == True]
presynaptic = cells_df[cells_df["is_starter"] == False].copy()
connectivity_matrix = presynaptic["unique_barcodes"].apply(
    lambda bcs: match_barcodes(starters["unique_barcodes"], bcs)
)
presynaptic["starters"] = connectivity_matrix.apply(
    lambda row: set(row.index[row]), axis=1
)
presynaptic["n_starters"] = presynaptic["starters"].apply(len)

In [ ]:
if False:
    save_path = main_path / "home/shared/transfer/for_alexk"
    save_path.mkdir(exist_ok=True)

    with pd.ExcelWriter(save_path / "brisc.xlsx") as writer:
        starters.to_excel(writer, sheet_name="Starters")
        presynaptic.to_excel(writer, sheet_name="Presynaptic")

In [ ]:
multibarcoded_starters = bc_cells.analyze_multibarcoded_starters(cells_df)

all_shuffled_distances, all_starter_coords = (
    dist_cells.create_barcode_shuffled_nulls_parallel(cells_df, N_iter=20)
)

relative_presyn_coords_flatmap, distancess_flatmap, starters_df = (
    dist_cells.determine_presynaptic_distances(cells_df, col_prefix="flatmap_")
)
all_shuffled_distances_flatmap, _ = dist_cells.create_barcode_shuffled_nulls_parallel(
    cells_df, N_iter=20, col_prefix="flatmap_"
)

atlas_size = 10
bin_image = spatial.prepare_area_labels(atlas_size=10)

cortex_bin_image = spatial.prepare_area_labels(
    atlas_size=10,
    structures=[
        "root",
        "CTX",
        "MB",
        "DG",
        "DG-mo",
        "DG-sg",
        "SCdg",
        "SCdw",
        "SCig",
        "SCiw",
        "SCop",
        "SCsg",
        "SCzo",
        "PAG",
        "MRN",
        "TH",
        "RN",
        "VISp",
        "VISl",
        "VISli",
        "VISal",
        "VISam",
        "VISpm",
        "AUDp",
        "AUDd",
        "RSP",
        "TEa",
        "AUDpo",
        "AUDv",
        "ECT",
        "ENTl",
    ],
    xpos=861,
)

barcode_10bp = {}
for barcode in cells_df["unique_barcodes"].explode().unique():
    barcode_10bp[barcode[:10]] = barcode

In [ ]:
fontsize_dict = {"title": 8, "label": 7, "tick": 6, "legend": 6}
pad_dict = {"label": 1, "tick": 1, "legend": 5}
line_width = 0.9
line_alpha = 1
kde_bw = 0.2

cm = 1 / 2.54
fig = plt.figure(figsize=(17.4 * cm, 17.4 * cm), dpi=300)
data_path = (
    main_path / "home/shared/projects/barcode_diversity_analysis/collapsed_barcodes/"
)
save_path = main_path / "home/shared/presentations/becalick_2025"

save_fig = True
figname = "fig4_presynaptic_networks_raster"

ax_coronal = fig.add_axes([-0.01, 0.75, 0.31, 0.22])
ax_flatmap = fig.add_axes([0.28, 0.83, 0.23, 0.12])

spatial.plot_all_rv_cells(
    cells_df,
    ax_coronal,
    ax_flatmap,
    bin_image,
    legend_fontsize=6,
    atlas_size=10,
    area_colors={
        "AUDp": "limegreen",
        "AUDpo": "mediumseagreen",
        "AUDv": "springgreen",
        "RSP": "darkorchid",
        "TEa": "forestgreen",
        "ECT": "darkolivegreen",
        "TH": "orangered",
        "VISal": "aquamarine",
        "VISl": "darkturquoise",
        "VISli": "mediumaquamarine",
        "VISp": "deepskyblue",
        "VISpm": "royalblue",
    },
    rasterized=True,
)

# ax_v1_rabies = fig.add_axes([0.60, 0.78, 0.1, 0.15])
ax_density = fig.add_axes([0.60, 0.78, 0.08, 0.15])
spatial.plot_layer_distribution(
    None,
    ax_density,
    cells_df,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    show_cells=False,
)

ax_starters_per_presynaptic = fig.add_axes([0.77, 0.78, 0.08, 0.15])
bc_cells.plot_hist(
    presynaptic,
    col="n_starters",
    ax=ax_starters_per_presynaptic,
    tick_fontsize=fontsize_dict["tick"],
    show_zero=False,
    show_counts=True,
    max_val=4,
)
ax_starters_per_presynaptic.set_ylabel(
    "Proportion of cells", fontsize=fontsize_dict["label"]
)
ax_starters_per_presynaptic.set_xlabel(
    "Connected starters", fontsize=fontsize_dict["label"]
)

ax_stack = fig.add_axes([0.93, 0.78, 0.10, 0.15])
bc_cells.plot_multibarcoded_starters(
    multibarcoded_starters,
    ax_stack,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    legend_fontsize=fontsize_dict["legend"],
    barcode_proportion=False,
)

barcodes = [
    "TGTCACTTGCATCC",
    "GCTTCATGCAATTG",
    # "TACAGCCAAATTTC",
    "GCTCTTCCTTAATA",
    "ATAAATAAGGCGCT",
]
ax_coronal_examples = fig.add_axes([0.0, 0.41, 0.4, 0.33])
ax_flatmap_examples = fig.add_axes([0.41, 0.44, 0.33, 0.33])

spatial.plot_example_barcodes(
    cells_df,
    ax_coronal_examples,
    ax_flatmap_examples,
    bin_image,
    barcodes=barcodes,
    barcode_colors=["crimson", "dodgerblue", "darkorange", "forestgreen"],
    legend_fontsize=6,
    atlas_size=10,
    starter_marker_size=25,
    presynaptic_marker_size=3,
    rasterized=True,
)

ax_presyn_shared = fig.add_axes([0.83, 0.62, 0.17, 0.12])

dist_cells.plot_relative_coors(
    relative_presyn_coords_flatmap / 100,
    ax=ax_presyn_shared,
    s=1,
    alpha=0.05,
    color="black",
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    coors_to_plot=(0, 2),
    lims=((-5, 5), (-1, 1)),
    labels=("", ""),
    rasterized=True,
)

ax_presyn_shared.set_xticklabels([])

ax_presyn_shuffled = fig.add_axes([0.83, 0.55, 0.17, 0.12])
dist_cells.plot_relative_coors(
    all_shuffled_distances_flatmap[0] / 100,
    ax=ax_presyn_shuffled,
    s=1,
    alpha=0.05,
    color="black",
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    coors_to_plot=(0, 2),
    lims=((-5, 5), (-1, 1)),
    labels=("", "            Relative cortical\n              depth (mm)"),
    rasterized=True,
)
ax_presyn_shuffled.text(
    5, 1, "Shuffle", ha="right", va="bottom", fontsize=fontsize_dict["legend"]
)

ax_presyn_shuffled.set_xticklabels([])

ax_kdeplot = fig.add_axes([0.83, 0.46, 0.17, 0.10])
sns.kdeplot(
    relative_presyn_coords_flatmap[:, 0] / 100,
    label="Observed",
    ax=ax_kdeplot,
    color="black",
    linewidth=0.9,
    bw_adjust=0.3,
)
sns.kdeplot(
    all_shuffled_distances_flatmap[0][:, 0] / 100,
    label="Shuffle",
    ax=ax_kdeplot,
    color="black",
    linewidth=0.9,
    linestyle=":",
    bw_adjust=0.3,
)
ax_kdeplot.set_xlabel("Relative M-L location (mm)", fontsize=fontsize_dict["label"])
ax_kdeplot.set_ylabel("Density", fontsize=fontsize_dict["label"])

plt.xlim(-5, 5)
sns.despine()
ax_kdeplot.legend(
    loc="lower right",
    fontsize=fontsize_dict["legend"],
    frameon=False,
    bbox_to_anchor=(1.1, 0.6),
    handlelength=1,
    handletextpad=0.5,
)
ax_kdeplot.tick_params(axis="both", which="major", labelsize=fontsize_dict["tick"])
ax_kdeplot.set_yticks([0, 0.5])

if save_fig:
    fig.savefig(save_path / f"{figname}.pdf", format="pdf", dpi=600)
    fig.savefig(save_path / f"{figname}.png", format="png")

In [ ]:
# Print useful numbers
# number of starter in v1
print(f"{(starters.cortical_area == 'VISp').mean()*100:.1f}% of starters are in V1")
print(
    f"Average of {connectivity_matrix.sum(axis=0).mean():.1f} presynaptic cells per starter"
)
print(
    f"{connectivity_matrix.shape[1]} starter neurons and {connectivity_matrix.values.sum()} connections"
)